# Use-Case for Data Scientist Interview at DeFiYield
Created by Kaan Korkmaz.

Here is our Rect Database: https://defiyield.app/rekt-database
Task: 
-  prepare the visualization of risk vectors and how it evolves over time?
- Share a jupyter notebook (or equivalent) in case of this task.

We expected just a few visualizations to highlight what are the most important insights from our database. 

The API key: bad7014e-3fd4-473a-b981-1a5c2dd2d72e

The doc about the API:
https://docs.defiyield.app/api/api

In [181]:
import json
import requests
import keyring
import pandas as pd
from bs4 import BeautifulSoup
import plotly.graph_objects as go

## Data Collection

In [2]:
api_key = keyring.get_password('defiyield', 'api_key')
headers = {'X-Api-Key': api_key}      
query_url = 'https://public-api.defiyield.app/graphql/'

In [3]:
def create_body(page_number):
  def body(): """query {
      rekts(
        pageNumber:%s
        pageSize:50
        orderBy: {
          date: desc
        }
      ) {
        id
        projectName
        description
        date
        fundsLost
        fundsReturned
        category
        chaindIds
        issueType
      }
    }"""
    
  def create_body_w_pageNumber(page_number):
    body.__doc__
    body.__doc__ %= page_number
  
  body()
  create_body_w_pageNumber(page_number)
  return body.__doc__

In [ ]:
data_list = []
for i in range(1, 27):
    print(i)
    body = create_body(i)
    
    response = requests.post(url=query_url,
                             json={'query': body},
                             headers=headers)
    
    data_list.append(response.json()["data"]["rekts"])

In [5]:
flat_list = [item for sublist in data_list for item in sublist]
df_rects = pd.DataFrame(flat_list)

## Preprocessing

In [6]:
df_rects['fundsLost'] = pd.to_numeric(df_rects['fundsLost'])
df_rects['fundsReturned'] = pd.to_numeric(df_rects['fundsReturned'])
df_rects['date'] = pd.to_datetime(df_rects['date'])
df_rects.head()

,id,projectName,description,date,fundsLost,fundsReturned,category,chaindIds,issueType
0,3320,Thoreum Finance,<p><strong>Quick Summary</strong></p><p>The Th...,2023-01-19,580000,0,Other,[2],Access Control
1,3319,BONK Inu (Fake),<p><strong>Quick Summary</strong></p><p>BONK I...,2023-01-18,150000,0,Token,[3],Rugpull
2,3321,OMNI Real Estate,<p><strong>Quick Summary</strong></p><p>The OM...,2023-01-17,70000,0,NFT,[2],Other
3,3316,Midas Capital,<p><strong>Quick Summary</strong></p><p>Midas ...,2023-01-15,650000,0,Borrowing and Lending,[3],Flash Loan Attack
4,3315,LendHub,<p><strong>Quick Summary</strong></p><p>The Le...,2023-01-12,6000000,0,Borrowing and Lending,[11],Other


In [85]:
df_rects.to_csv('data/all_data.csv')

In [102]:
# Create df per issueType for visualization purposes
df_rects_issue_abandoned = df_rects[df_rects["issueType"] == "Abandoned"]
df_rects_issue_honeypot = df_rects[df_rects["issueType"] == "Honeypot"]
df_rects_issue_rugpull = df_rects[df_rects["issueType"] == "Rugpull"]
df_rects_issue_accesscontrol = df_rects[df_rects["issueType"] == "Access Control"]
df_rects_issue_phishing = df_rects[df_rects["issueType"] == "Phishing"]
df_rects_issue_flashloanattack = df_rects[df_rects["issueType"] == "Flash Loan Attack"]
df_rects_issue_reentrancy = df_rects[df_rects["issueType"] == "Reentrancy"]
df_rects_issue_oracleissue = df_rects[df_rects["issueType"] == "Oracle Issue"]
df_rects_issue_other = df_rects[df_rects["issueType"] == "Other"]

In [103]:
# Save
df_rects_issue_abandoned.to_csv('data/issue_abandoned.csv')
df_rects_issue_honeypot.to_csv('data/issue_honeypot.csv')
df_rects_issue_rugpull.to_csv('data/issue_rugpull.csv')
df_rects_issue_accesscontrol.to_csv('data/issue_accesscontrol.csv')
df_rects_issue_phishing.to_csv('data/issue_phishing.csv')
df_rects_issue_flashloanattack.to_csv('data/issue_flashloanattack.csv')
df_rects_issue_reentrancy.to_csv('data/issue_reentrancy.csv')
df_rects_issue_oracleissue.to_csv('data/issue_oracleissue.csv')
df_rects_issue_other.to_csv('data/issue_other.csv')

In [38]:
dfff_ = df_rects.resample(rule='M', on='date')['fundsLost'].sum().reset_index()

In [112]:
dfff = pd.DataFrame(dfff_, )
dfff.to_csv('data/grouped_by_day_sum_fundsLost.csv')

In [113]:
dfff.head()

,date,fundsLost
0,2011-06-30,8818300
1,2011-07-31,254100
2,2011-08-31,619000
3,2011-09-30,32800
4,2011-10-31,61120


In [135]:
df_groupby_date_issueType = df_rects.groupby(by=['date', 'issueType'], as_index=False)['fundsLost'].sum()
df_groupby_date_issueType.to_csv('data/grouped_by_date_issueType_sum_fundsLost.csv')

In [280]:
df_groupby_date_issueType = df_rects.groupby(by=['date', 'category'], as_index=False)['fundsLost'].sum()
df_groupby_date_issueType.to_csv('data/grouped_by_date_category_sum_fundsLost.csv')

In [ ]:
df_groupby_date_issueType.head()

,date,issueType,fundsLost
0,2011-06-13,Other,8750000
1,2011-06-21,Other,68300
2,2011-07-01,Rugpull,18100
3,2011-07-26,Access Control,236000
4,2011-08-08,Rugpull,619000


In [119]:
df_groupby_date_issueType_pivoted = df_groupby_date_issueType.pivot(index='date', 
                                                                    columns='issueType', 
                                                                    values='fundsLost').fillna(0).rename_axis(None, axis=1)

# df_groupby_date_issueType_pivoted.columns=df_groupby_date_issueType_pivoted.columns.get_level_values(0)
# df_groupby_date_issueType_pivoted = df_groupby_date_issueType_pivoted.xs('issueType', axis=1, drop_level=True)


In [137]:
df_groupby_date_issueType_pivoted.to_csv('data/per_day_issueType.csv')

In [138]:
df_groupby_date_issueType_pivoted[0:40]

,Abandoned,Access Control,Flash Loan Attack,Honeypot,Oracle Issue,Other,Phishing,Reentrancy,Rugpull
date,,,,,,,,,
2011-06-13,0.0,0.0,0.0,0.0,0.0,8.750000e+06,0.0,0.0,0.0
2011-06-21,0.0,0.0,0.0,0.0,0.0,6.830000e+04,0.0,0.0,0.0
2011-07-01,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,18100.0
2011-07-26,0.0,236000.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0
2011-08-08,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,619000.0
2011-09-01,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,32800.0
2011-10-05,0.0,0.0,0.0,0.0,0.0,5.310000e+04,0.0,0.0,0.0
2011-10-28,0.0,0.0,0.0,0.0,0.0,8.020000e+03,0.0,0.0,0.0
2012-03-01,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,6450.0


In [139]:
# df_groupby_date_issueType_pivoted = df_groupby_date_issueType_pivoted.set_index('date') 
df_groupby_date_issueType_pivoted_month = df_groupby_date_issueType_pivoted.groupby(pd.Grouper(freq='M')).aggregate('sum')
df_groupby_date_issueType_pivoted_month.to_csv('data/per_month_issueType.csv')


In [149]:
df_groupby_date_issueType_pivoted_month_count = df_groupby_date_issueType_pivoted.groupby(pd.Grouper(freq='M')).aggregate('count')

In [150]:
df_groupby_date_issueType_pivoted_month_count

,Abandoned,Access Control,Flash Loan Attack,Honeypot,Oracle Issue,Other,Phishing,Reentrancy,Rugpull
date,,,,,,,,,
2011-06-30,2,2,2,2,2,2,2,2,2
2011-07-31,2,2,2,2,2,2,2,2,2
2011-08-31,1,1,1,1,1,1,1,1,1
2011-09-30,1,1,1,1,1,1,1,1,1
2011-10-31,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...
2022-09-30,19,19,19,19,19,19,19,19,19
2022-10-31,25,25,25,25,25,25,25,25,25
2022-11-30,16,16,16,16,16,16,16,16,16


In [134]:
df_groupby_date_issueType_pivoted_month[df_groupby_date_issueType_pivoted_month.index == '2013-04-30']

,Abandoned,Access Control,Flash Loan Attack,Honeypot,Oracle Issue,Other,Phishing,Reentrancy,Rugpull
date,,,,,,,,,
2013-04-30,0.0,371000.0,0.0,0.0,0.0,135000.0,0.0,0.0,0.0


# Visualization

Show how risk vectors evolve over time.

In [13]:
import plotly.express as px
# df = px.data.stocks()
fig = px.line(df_rects, x="date", y=df_rects["fundsLost"],
            #   hover_data={"date": "|%B %d, %Y"},
              title='custom tick labels with ticklabelmode="period"')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

In [65]:
import plotly.express as px
# df = px.data.stocks()
fig = px.line(df_groupby_date_issueType_pivoted, x="date", y=df_groupby_date_issueType_pivoted['Abandoned'],
            #   hover_data={"date": "|%B %d, %Y"},
              title='custom tick labels with ticklabelmode="period"')
# fig.update_xaxes(
#     dtick="M1",
#     tickformat="%Y",
#     ticklabelmode="period")
fig.show()

In [ ]:
fig = px.bar(df_groupby_date_issueType_pivoted_month, x="medal", y="count", color="nation", text="nation")
fig.show()

In [141]:
df = px.data.stocks()

In [142]:
df.head()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708


In [153]:
import plotly.express as px

fig = px.bar(df_groupby_date_issueType_pivoted, 
              x=df_groupby_date_issueType_pivoted.index, 
              y=df_groupby_date_issueType_pivoted.columns,
            #   hover_data={"date": "|%B %d, %Y"},
              title='Issue Type Trend over Time')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

In [154]:
import plotly.express as px

wide_df = px.data.medals_wide()

fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()

In [155]:
wide_df

,nation,gold,silver,bronze
0,South Korea,24,13,11
1,China,10,15,8
2,Canada,9,12,12


In [158]:
import plotly.express as px
fig = px.histogram(df_rects, x="date", color="issueType")
fig.show()

In [163]:
fig = px.histogram(df_rects, x="date", color="category").update_yaxes(categoryorder="total ascending")
fig.show()

In [164]:
df_rects.sort_values(["category", "issueType"])

,id,projectName,description,date,fundsLost,fundsReturned,category,chaindIds,issueType,percent_fundsLost
399,2691,bZx (4),"<p>The private key, controlling BSC and Polygo...",2021-11-05,47600000,0,Borrowing and Lending,"[2, 3]",Access Control,0.063841
875,2705,Wild Credit,<p>The attacker's address:<br>https://ethersca...,2021-05-27,611963,611963,Borrowing and Lending,[1],Access Control,0.000821
897,2365,EasyFi,<p><strong>Quick Summary</strong></p><p>EasyFi...,2021-04-19,59000000,0,Borrowing and Lending,[1],Access Control,0.079130
3,3316,Midas Capital,<p><strong>Quick Summary</strong></p><p>Midas ...,2023-01-15,650000,0,Borrowing and Lending,[3],Flash Loan Attack,0.000872
21,3288,Lodestar Finance,<p><strong>Quick Summary</strong></p><p>Lodest...,2022-12-10,5800000,0,Borrowing and Lending,[5],Flash Loan Attack,0.007779
...,...,...,...,...,...,...,...,...,...,...
1199,2901,Netbox,"<p>The Etbox platform wallet was hacked, causi...",2019-03-24,132000,0,None,[],Rugpull,0.000177
1240,2671,OneCoin,<p>OneCoin was a Ponzi scheme that pulled in $...,2016-03-07,4000000,0,None,[],Rugpull,0.005365
1268,2955,BASIC-MINING,<p>Mining company BASIC-MINING took advantage ...,2013-10-01,277000,0,None,[],Rugpull,0.000372
1286,2975,Bitcoin Savings and Trust,<p>Bitcoin Savings &amp; Trust (abbreviated as...,2012-07-02,4630000,0,None,[],Rugpull,0.006210


In [228]:
new_df = df_rects.groupby(by=["category", "issueType"]).aggregate('fundsLost').sum()
new_df.to_csv('data/heatmap.csv')

In [177]:
dfff = pd.read_csv('data/heatmap.csv')

In [210]:
dfff = dfff.sort_values(["category", "issueType"], ascending=False)

In [211]:
x_categories = dfff['issueType']
y_categories = dfff['category']

In [225]:
fig = go.Figure(data=go.Heatmap(
                   z=dfff['fundsLost'],
                   x=x_categories,
                   y=y_categories,
                   hoverongaps = False,
                   colorscale='balance'))

fig.layout.title = "Total Amount of Lost Funds ($)"
fig.layout.height = 1200
fig.layout.width = 1400
fig.update_xaxes(side="top")
fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig.show()

In [234]:
df_groupby_date_issueType.head()

,date,issueType,fundsLost
0,2011-06-13,Other,8750000
1,2011-06-21,Other,68300
2,2011-07-01,Rugpull,18100
3,2011-07-26,Access Control,236000
4,2011-08-08,Rugpull,619000


In [238]:
data_canada = px.data.gapminder().query("country == 'Canada'")


In [244]:
data_canada.dtypes
# fig = px.bar(data_canada, x='year', y='pop')
# fig.show()


country       object
continent     object
year           int64
lifeExp      float64
pop            int64
gdpPercap    float64
iso_alpha     object
iso_num        int64
dtype: object

In [246]:
df_groupby_date_issueType['year'] = df_groupby_date_issueType['date'].dt.year

In [250]:
df_groupby_date_issueType

,date,issueType,fundsLost,year
0,2011-06-13,Other,8750000,2011
1,2011-06-21,Other,68300,2011
2,2011-07-01,Rugpull,18100,2011
3,2011-07-26,Access Control,236000,2011
4,2011-08-08,Rugpull,619000,2011
...,...,...,...,...
800,2023-01-12,Other,6000000,2023
801,2023-01-15,Flash Loan Attack,650000,2023
802,2023-01-17,Other,70000,2023
803,2023-01-18,Rugpull,150000,2023


In [251]:
fig = px.bar(df_groupby_date_issueType, x="year", y="fundsLost", color="issueType" ,title="Long-Form Input")
fig.show()

In [252]:
ig = go.Figure(data=go.Bar(x=df_groupby_date_issueType['date'].astype(dtype=str), 
                        y=df_groupby_date_issueType['fundsLost'],
                        marker_color='issueType', text="counts"))
fig.update_layout({"title": 'Tweets about Malioboro from Jan 2020 to Jan 2021',
                   "xaxis": {"title":"Months"},
                   "yaxis": {"title":"Total tweets"},
                   "showlegend": False})

fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of bar.marker
        Received value: 'issueType'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deepskyb

In [267]:
df_groupby_date_issueType['year'] = pd.to_datetime(df_groupby_date_issueType['date']).dt.to_period('Y')
# by_month.index = pd.PeriodIndex(by_month.index)

# df_month = by_month.rename_axis('month').reset_index(name='counts')
df_groupby_date_issueType_new = df_groupby_date_issueType.groupby(by=["year", "issueType"]).aggregate({"fundsLost":"sum"}).reset_index()

In [278]:
df_groupby_date_issueType_new.head()

,year,issueType,fundsLost
0,2011,Access Control,236000
1,2011,Other,8879420
2,2011,Rugpull,669900
3,2012,Access Control,443300
4,2012,Other,3360619200


In [271]:
fig = px.bar(df_groupby_date_issueType, x="year", y="fundsLost", color="issueType" ,title="Long-Form Input")
fig.show()

TypeError: Object of type Period is not JSON serializable

In [269]:
fig = go.Figure(data=go.Bar(x=df_groupby_date_issueType_new['year'].astype(dtype=str), 
                        y=df_groupby_date_issueType_new['fundsLost'],
                        marker_color='indianred', text="counts"))

fig.update_layout({"title": 'Tweets about Malioboro from Jan 2020 to Jan 2021',
                   "xaxis": {"title":"Months"},
                   "yaxis": {"title":"Total tweets"},
                   "showlegend": False})
# fig.write_image("by-month-bar.png",format="png", width=1000, height=600, scale=3)
fig.show()

In [274]:
# df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(df_groupby_date_issueType, x="date", y="fundsLost", color='issueType')
fig.show()

In [273]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
60,Australia,Oceania,1952,69.12,8691212,10039.59564,AUS,36
61,Australia,Oceania,1957,70.33,9712569,10949.64959,AUS,36
62,Australia,Oceania,1962,70.93,10794968,12217.22686,AUS,36
63,Australia,Oceania,1967,71.10,11872264,14526.12465,AUS,36
64,Australia,Oceania,1972,71.93,13177000,16788.62948,AUS,36


In [283]:
list_of_categories = df_rects.category.unique()

In [287]:

res = [i for i in list_of_categories if i is not None]

In [288]:
sorted_list = sorted(res, key=lambda x: x[0]) 

In [293]:
sorted_list[20:]

['Gaming / Metaverse,Token',
 'Gaming / Metaverse',
 'NFT,Token',
 'NFT,Other',
 'NFT',
 'Other',
 'Other,Token',
 'Other,Yield Aggregator',
 'Stablecoin',
 'Stablecoin,Yield Aggregator',
 'Token,Yield Aggregator',
 'Token',
 'Yield Aggregator']

In [297]:
x = df_rects.issueType.unique()

In [298]:
x.sort()

In [299]:
x

array(['Abandoned', 'Access Control', 'Flash Loan Attack', 'Honeypot',
       'Oracle Issue', 'Other', 'Phishing', 'Reentrancy', 'Rugpull'],
      dtype=object)